# Tennis TrackNet 2x — Colab Training

Train **TrackNet2x** (576x1024 resolution) on the tennis dataset with GPU optimizations.

**Optimizations enabled on Colab:**
- Mixed precision (FP16) — ~2x speedup
- torch.compile — ~1.3x speedup
- Larger batch size (8-16 vs 2-4 locally)
- Precomputed frames for fast data loading

**Requirements:** GPU runtime (T4 or better), Google Drive mounted

## 1. Setup

In [3]:
# Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    mem_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f'Memory: {mem_gb:.1f} GB')
    # Recommend batch size based on GPU memory
    # batch_size=16 OOMs on A100 with torch.compile; 8 is safe
    if mem_gb >= 40:
        print('Recommended batch_size: 8')
    elif mem_gb >= 15:
        print('Recommended batch_size: 4')
    else:
        print('Recommended batch_size: 2')

NVIDIA A100-SXM4-80GB, 81920 MiB
PyTorch: 2.9.0+cu128
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
Memory: 85.1 GB
Recommended batch_size: 8


In [4]:
# Mount Google Drive (for persistent storage across sessions)
from google.colab import drive
drive.mount('/content/drive')

# Persistent storage dir
DRIVE_DIR = '/content/drive/MyDrive/tennis-tracknet'
!mkdir -p {DRIVE_DIR}

Mounted at /content/drive


In [5]:
# Clone private repo (will prompt for GitHub auth)
import os
if not os.path.exists('/content/tennis-tracknet'):
    !git clone https://github.com/smyng/tennis-tracknet.git /content/tennis-tracknet
else:
    !cd /content/tennis-tracknet && git pull

os.chdir('/content/tennis-tracknet')
!pwd

Cloning into '/content/tennis-tracknet'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 91 (delta 50), reused 69 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 88.37 KiB | 5.52 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/tennis-tracknet


In [6]:
# Install dependencies
!pip install -q parse tqdm tensorboard

## 2. Dataset

Downloads the TrackNet v1 tennis dataset and converts it to TrackNetV3 format.
Converted data is cached in Google Drive so you only do this once.

In [7]:
import os

DATA_DRIVE = f'{DRIVE_DIR}/data'
DATA_LOCAL = '/content/tennis-tracknet/data'

# Check if converted data already exists in Drive
if os.path.exists(f'{DATA_DRIVE}/train/match1'):
    print('Converted dataset found in Drive, symlinking...')
    !rm -rf {DATA_LOCAL}
    !ln -s {DATA_DRIVE} {DATA_LOCAL}
    !ls {DATA_LOCAL}/train/
    print('Done.')
else:
    print('No converted dataset in Drive. Will download and convert.')
    print('This takes ~15-20 min the first time.')

Converted dataset found in Drive, symlinking...
match1	match2	match3	match4	match5	match6	match7	match8
Done.


In [8]:
# Download raw tennis dataset (skip if data already linked above)
import os
if not os.path.exists(f'{DATA_LOCAL}/train/match1'):
    !pip install -q gdown
    
    RAW_DIR = '/content/raw-tennis-dataset'
    !mkdir -p {RAW_DIR}
    
    # Download from the TrackNet v1 dataset Google Drive
    # Folder: https://drive.google.com/drive/folders/11r0RUaQHX7I3ANkaYG4jOxXK1OYo01Ut
    import gdown
    gdown.download_folder(
        'https://drive.google.com/drive/folders/11r0RUaQHX7I3ANkaYG4jOxXK1OYo01Ut',
        output=RAW_DIR, quiet=False
    )
    
    # The dataset is inside Dataset.zip — unzip it
    import zipfile
    zip_path = os.path.join(RAW_DIR, 'Dataset.zip')
    if os.path.exists(zip_path):
        print('Extracting Dataset.zip...')
        with zipfile.ZipFile(zip_path, 'r') as z:
            z.extractall(RAW_DIR)
        os.remove(zip_path)
    
    # Find the directory containing game1/, game2/, etc.
    DATASET_DIR = RAW_DIR
    for candidate in [os.path.join(RAW_DIR, 'Dataset'), RAW_DIR]:
        if os.path.exists(os.path.join(candidate, 'game1')):
            DATASET_DIR = candidate
            break
    
    print(f'Dataset directory: {DATASET_DIR}')
    !ls {DATASET_DIR}/

In [9]:
# Convert dataset and save to Drive for persistence
import os
if not os.path.exists(f'{DATA_LOCAL}/train/match1'):
    # Find the dataset directory (set by previous cell, or detect it)
    if 'DATASET_DIR' not in dir():
        RAW_DIR = '/content/raw-tennis-dataset'
        DATASET_DIR = os.path.join(RAW_DIR, 'Dataset') if os.path.exists(os.path.join(RAW_DIR, 'Dataset', 'game1')) else RAW_DIR

    !python scripts/convert_tennis_dataset.py \
        --input {DATASET_DIR} \
        --output {DATA_DRIVE} \
        --test-games 9 10 \
        --verbose
    
    # Symlink Drive data into repo
    !rm -rf {DATA_LOCAL}
    !ln -s {DATA_DRIVE} {DATA_LOCAL}
    
    print('Conversion complete.')
    !ls {DATA_LOCAL}/train/

In [10]:
# Generate median.npz files (required by dataset.py for background subtraction)
# Saved to Drive, so this only runs once
import os, cv2, numpy as np
from pathlib import Path
from tqdm import tqdm

data_dir = DATA_DRIVE
needs_generation = False

# Check if median files already exist
for split in ['train', 'test']:
    split_dir = Path(data_dir) / split
    if not split_dir.exists():
        continue
    for match_dir in split_dir.iterdir():
        frame_root = match_dir / 'frame'
        if not frame_root.exists():
            continue
        for rally_dir in frame_root.iterdir():
            if rally_dir.is_dir() and not (rally_dir / 'median.npz').exists():
                needs_generation = True
                break
        if needs_generation:
            break
    if needs_generation:
        break

if needs_generation:
    print('Generating median.npz files (one-time)...')
    for split in ['train', 'test']:
        split_dir = Path(data_dir) / split
        if not split_dir.exists():
            continue
        for match_dir in sorted(split_dir.iterdir()):
            if not match_dir.is_dir():
                continue
            frame_root = match_dir / 'frame'
            if not frame_root.exists():
                continue
            rally_medians = []
            for rally_dir in tqdm(sorted(frame_root.iterdir()), desc=f'{split}/{match_dir.name}'):
                if not rally_dir.is_dir():
                    continue
                median_file = rally_dir / 'median.npz'
                if median_file.exists():
                    rally_medians.append(np.load(str(median_file))['median'])
                    continue
                frames = sorted(rally_dir.glob('*.png'))
                if not frames:
                    continue
                step = max(1, len(frames) // 50)
                sampled = frames[::step][:50]
                imgs = [cv2.imread(str(f))[..., ::-1] for f in sampled]
                median = np.median(np.array(imgs), axis=0)
                np.savez(str(median_file), median=median)
                rally_medians.append(median)
            # Match-level median
            match_median = match_dir / 'median.npz'
            if not match_median.exists() and rally_medians:
                median = np.median(np.array(rally_medians), axis=0)
                np.savez(str(match_median), median=median)
    print('Done.')
else:
    print('median.npz files already exist.')

median.npz files already exist.


## 3. Pretrained Weights

Download the original TrackNetV3 badminton (shuttlecock) checkpoint.

In [11]:
import os

CKPT_DIR = '/content/tennis-tracknet/ckpts'
os.makedirs(CKPT_DIR, exist_ok=True)

if not os.path.exists(f'{CKPT_DIR}/TrackNet_best.pt'):
    !pip install -q gdown
    import gdown
    
    # Original TrackNetV3 checkpoints
    # https://drive.google.com/file/d/1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA/view
    gdown.download(
        'https://drive.google.com/uc?id=1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA',
        output='/content/TrackNetV3_ckpts.zip', quiet=False
    )
    !cd /content && unzip -o TrackNetV3_ckpts.zip -d /content/ckpts_tmp/
    !rm /content/TrackNetV3_ckpts.zip
    
    # The zip contains a nested ckpts/ folder — flatten it
    import glob, shutil
    for pt_file in glob.glob('/content/ckpts_tmp/**/*.pt', recursive=True):
        shutil.move(pt_file, CKPT_DIR)
    !rm -rf /content/ckpts_tmp

print('Pretrained checkpoints:')
!ls -lh {CKPT_DIR}/*.pt

# Verify checkpoint
import torch
ckpt = torch.load(f'{CKPT_DIR}/TrackNet_best.pt', map_location='cpu', weights_only=False)
print(f"\nPretrained model: epoch {ckpt['epoch']}, bg_mode='{ckpt['param_dict']['bg_mode']}'")
print(f"Original training: seq_len={ckpt['param_dict']['seq_len']}, batch_size={ckpt['param_dict']['batch_size']}")

Downloading...
From (original): https://drive.google.com/uc?id=1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA
From (redirected): https://drive.google.com/uc?id=1CfzE87a0f6LhBp0kniSl1-89zaLCZ8cA&confirm=t&uuid=f55a0403-da02-44c2-b2d6-0bea5276ce44
To: /content/TrackNetV3_ckpts.zip
100%|██████████| 132M/132M [00:01<00:00, 102MB/s]  


Archive:  TrackNetV3_ckpts.zip
   creating: /content/ckpts_tmp/ckpts/
  inflating: /content/ckpts_tmp/ckpts/InpaintNet_best.pt  
  inflating: /content/ckpts_tmp/ckpts/TrackNet_best.pt  
Pretrained checkpoints:
-rw-r--r-- 1 root root 6.0M Aug  8  2023 /content/tennis-tracknet/ckpts/InpaintNet_best.pt
-rw-r--r-- 1 root root 130M Aug  8  2023 /content/tennis-tracknet/ckpts/TrackNet_best.pt

Pretrained model: epoch 18, bg_mode='concat'
Original training: seq_len=8, batch_size=10


## 4. Precompute Frames (one-time)

Precomputes resized + background-subtracted frames as `.npy` files for fast data loading.
Cached in Google Drive so this only runs once (~20 min).

In [12]:
# Precompute frames (skip if already done)
# Since data/ is symlinked to Drive, output goes to Drive automatically
# Uses parallel workers + threaded I/O for ~3-4x speedup on Drive
import os, glob

PRECOMPUTE_DIR = os.path.join(DATA_LOCAL, 'precomputed', 'subtract_concat_576x1024')

existing = glob.glob(os.path.join(PRECOMPUTE_DIR, '*.npy'))
if len(existing) > 90:
    print(f'Precomputed frames found: {len(existing)} files — skipping')
else:
    print(f'Precomputing frames ({len(existing)} found, need ~95)...')
    !python precompute_frames.py \
        --data_dir {DATA_LOCAL} \
        --bg_mode subtract_concat \
        --height 576 --width 1024 \
        --workers 4 --io_threads 8 \
        --splits train val

Precomputed frames found: 95 files — skipping


In [13]:
# Copy precomputed frames to local SSD for ~10x faster data loading
# Google Drive FUSE is slow for large file reads — local SSD eliminates the I/O bottleneck
# This takes ~5-10 min but saves significant time per epoch
import os, time

PRECOMPUTE_LOCAL = '/content/precomputed_local'

if not os.path.exists(PRECOMPUTE_LOCAL):
    print('Copying precomputed frames from Drive to local SSD...')
    start = time.time()
    !cp -r {DATA_DRIVE}/precomputed {PRECOMPUTE_LOCAL}
    elapsed = time.time() - start
    print(f'Done in {elapsed/60:.1f} min')
    !du -sh {PRECOMPUTE_LOCAL}
else:
    print('Local precomputed copy already exists')
    !du -sh {PRECOMPUTE_LOCAL}

# Set env var so dataset.py reads precomputed from local SSD instead of Drive
os.environ['PRECOMPUTED_DIR'] = PRECOMPUTE_LOCAL
print(f'PRECOMPUTED_DIR={PRECOMPUTE_LOCAL}')

Copying precomputed frames from Drive to local SSD...
Done in 13.2 min
44G	/content/precomputed_local
PRECOMPUTED_DIR=/content/precomputed_local


In [14]:
# Upload local checkpoint to resume training on Colab
# Run this cell, then select your TrackNet2x_cur.pt file from local machine
import os
from google.colab import files

EXP_NAME = 'tennis_2x_colab'
SAVE_DIR = f'{DRIVE_DIR}/exps/{EXP_NAME}'
os.makedirs(SAVE_DIR, exist_ok=True)

# Symlink exps into repo
!mkdir -p {DRIVE_DIR}/exps
!rm -rf /content/tennis-tracknet/exps
!ln -s {DRIVE_DIR}/exps /content/tennis-tracknet/exps

# Upload checkpoint
# print('Upload TrackNet2x_cur.pt (and optionally TrackNet2x_best.pt):')
# uploaded = files.upload()

# for name, data in uploaded.items():
#     dest = os.path.join(SAVE_DIR, name)
#     with open(dest, 'wb') as f:
#         f.write(data)
#     print(f'Saved {name} to {dest} ({len(data)/1e6:.1f} MB)')

# Verify
import torch
ckpt = torch.load(os.path.join(SAVE_DIR, 'TrackNet2x_cur.pt'), map_location='cpu', weights_only=False)
print(f"\nCheckpoint: epoch {ckpt['epoch']+1}, best_val_acc={ckpt['max_val_acc']:.4f}")


Checkpoint: epoch 5, best_val_acc=1.0000


## 5. Train

**Option A:** Start fresh training from scratch (cell below).
**Option B:** Resume from a local checkpoint — upload it first, then use the resume cell.

In [15]:
# Training config (used by both fresh start and resume)
import os, torch, subprocess

EXP_NAME = 'tennis_2x_colab'
SAVE_DIR = f'{DRIVE_DIR}/exps/{EXP_NAME}'
os.makedirs(SAVE_DIR, exist_ok=True)

# Symlink exps into repo so train.py can find them
!mkdir -p {DRIVE_DIR}/exps
!rm -rf /content/tennis-tracknet/exps
!ln -s {DRIVE_DIR}/exps /content/tennis-tracknet/exps

# Kill any zombie GPU processes from interrupted training runs
try:
    result = subprocess.run(
        ['nvidia-smi', '--query-compute-apps=pid', '--format=csv,noheader'],
        capture_output=True, text=True)
    my_pid = str(os.getpid())
    for pid in result.stdout.strip().split('\n'):
        pid = pid.strip()
        if pid and pid != my_pid:
            subprocess.run(['kill', '-9', pid])
            print(f'Killed zombie GPU process {pid}')
except Exception:
    pass

# Batch size: 8 for A100 (16 OOMs with torch.compile), 4 for T4/V100
mem_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
BATCH_SIZE = 8 if mem_gb >= 40 else (4 if mem_gb >= 15 else 2)

print(f'Experiment: {EXP_NAME}')
print(f'Batch size: {BATCH_SIZE} (GPU: {mem_gb:.0f} GB)')
print(f'Checkpoints: {SAVE_DIR}')

# Check for existing checkpoint
ckpt_path = os.path.join(SAVE_DIR, 'TrackNet2x_cur.pt')
if os.path.exists(ckpt_path):
    ckpt = torch.load(ckpt_path, map_location='cpu', weights_only=False)
    print(f"Found checkpoint: epoch {ckpt['epoch']+1}, best_val_acc={ckpt['max_val_acc']:.4f}")
    print('Use the RESUME cell below to continue training.')
else:
    print('No checkpoint found. Use the FRESH START cell below.')

Experiment: tennis_2x_colab
Batch size: 8 (GPU: 85 GB)
Checkpoints: /content/drive/MyDrive/tennis-tracknet/exps/tennis_2x_colab
Found checkpoint: epoch 5, best_val_acc=1.0000
Use the RESUME cell below to continue training.


In [ ]:
# FRESH START — train from scratch (no checkpoint needed)
# Delete any corrupted checkpoints first
import os
for f in ['TrackNet2x_cur.pt', 'TrackNet2x_best.pt']:
    p = os.path.join(SAVE_DIR, f)
    if os.path.exists(p):
        os.remove(p)
        print(f'Removed {f}')

!python train.py \
    --model_name TrackNet2x \
    --epochs 30 \
    --batch_size {BATCH_SIZE} \
    --bg_mode subtract_concat \
    --height 576 --width 1024 \
    --fp16 \
    --compile \
    --num_workers 2 \
    --frame_alpha 0.5 \
    --save_dir exps/{EXP_NAME} \
    --verbose

In [ ]:
# RESUME — continue training from checkpoint
# Only use this if you have a valid checkpoint in SAVE_DIR
!python train.py \
    --model_name TrackNet2x \
    --epochs 30 \
    --batch_size {BATCH_SIZE} \
    --bg_mode subtract_concat \
    --height 576 --width 1024 \
    --fp16 \
    --compile \
    --num_workers 2 \
    --frame_alpha 0.5 \
    --save_dir exps/{EXP_NAME} \
    --resume_training \
    --verbose

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# TensorBoard
EXP_NAME = 'tennis_2x_colab'
%load_ext tensorboard
%tensorboard --logdir {DRIVE_DIR}/exps/{EXP_NAME}/logs

<IPython.core.display.Javascript object>

In [ ]:
# Evaluate best model on test set
EXP_NAME = 'tennis_2x_colab'

!python test.py \
    --split test \
    --tracknet_file exps/{EXP_NAME}/TrackNet2x_best.pt \
    --save_dir exps/{EXP_NAME}/eval

Loading checkpoint...
Traceback (most recent call last):
  File "/content/tennis-tracknet/test.py", line 938, in <module>
    tracknet_ckpt = torch.load(args.tracknet_file, weights_only=False)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 1530, in load
    return _load(
           ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 2122, in _load
    result = unpickler.load()
             ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 2086, in persistent_load
    typed_storage = load_tensor(
                    ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 2052, in load_tensor
    wrap_storage = restore_location(storage, location)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line

In [ ]:
# Copy best model back to Drive for download
EXP_NAME = 'tennis_2x_colab'
!cp exps/{EXP_NAME}/TrackNet2x_best.pt {DRIVE_DIR}/{EXP_NAME}_best.pt
print(f'Best model saved to: {DRIVE_DIR}/{EXP_NAME}_best.pt')
print('You can download it from Google Drive.')

Best model saved to: /content/drive/MyDrive/tennis-tracknet/tennis_2x_colab_best.pt
You can download it from Google Drive.
